# Pretatamiento de datos
### Conversion de variables categoricas a numericas


In [3]:
import pandas as pd

df = pd.read_csv('Titanic.csv')

In [5]:
## Pretatamiento de datos
from sklearn.preprocessing import StandardScaler, LabelEncoder
titanic_data = df.copy()
# 1. Llenar valores faltantes
titanic_data['Age'].fillna(titanic_data['Age'].median(), inplace=True)
titanic_data['Embarked'].fillna('S', inplace=True)

# 2. Eliminar columnas innecesarias
titanic_data.drop(columns=['Cabin', 'Name', 'Ticket', 'PassengerId'], inplace=True)

# 3. Convertir variables categóricas a numéricas
label_encoder = LabelEncoder()
titanic_data['Sex'] = label_encoder.fit_transform(titanic_data['Sex'])  # male = 1, female = 0
titanic_data['Embarked'] = label_encoder.fit_transform(titanic_data['Embarked'])  # S=2, C=0, Q=1

# 4. Estandarizar los datos
scaler = StandardScaler()
scaled_data = scaler.fit_transform(titanic_data)


C:\Users\santi\AppData\Local\Temp\ipykernel_17332\2870053945.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  titanic_data['Age'].fillna(titanic_data['Age'].median(), inplace=True)
C:\Users\santi\AppData\Local\Temp\ipykernel_17332\2870053945.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behav